In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error

data = pd.read_csv('heart.csv')

In [6]:
from sklearn.model_selection import train_test_split

X = data.drop(['HeartDisease'], axis=1)
y = data.HeartDisease
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size=0.75,test_size=0.25)

In [16]:
from sklearn.preprocessing import OneHotEncoder
OHE = OneHotEncoder(handle_unknown='ignore',sparse=False)
categorical_cols = [cols for cols in X.columns if X_train[cols].dtype=='object']
num_cols = [cols for cols in X.columns if X_train[cols].dtype in ['int64','float64']]
#print(categorical_col)
#print(num_cols)
cat_X_train = X_train.drop(num_cols, axis=1)
cat_X_valid = X_valid.drop(num_cols, axis=1)
num_X_train = X_train.drop(categorical_cols, axis=1)
num_X_valid = X_valid.drop(categorical_cols, axis=1)
OH_cols_train = pd.DataFrame(OHE.fit_transform(cat_X_train) )
OH_cols_valid = pd.DataFrame(OHE.fit_transform(cat_X_valid))
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index
OH_X_train = pd.concat([num_X_train,OH_cols_train],axis=1)
OH_X_valid = pd.concat([num_X_valid,OH_cols_valid],axis=1)

In [19]:
from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=5000, learning_rate=0.02, n_jobs=4)
model.fit(OH_X_train,y_train,early_stopping_rounds=5,verbose=False, eval_set=[(OH_X_valid,y_valid)])
preds = model.predict(OH_X_valid)
mae = mean_absolute_error(preds, y_valid)
print(mae)

0.22192783318459988
